In [1]:
%pip install flask pandas pyglet pyrr numpy

   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 10.2/984.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/984.0 kB 435.7 kB/s eta 0:00:03
   - ------------------------------------- 41.0/984.0 kB 393.8 kB/s eta 0:00:03
   -- ------------------------------------ 71.7/984.0 kB 491.5 kB/s eta 0:00:02
   --- ----------------------------------- 92.2/984.0 kB 525.1 kB/s eta 0:00:02
   ---- --------------------------------- 122.9/984.0 kB 554.9 kB/s eta 0:00:02
   ----- -------------------------------- 153.6/984.0 kB 612.6 kB/s eta 0:00:02
   ------- ------------------------------ 184.3/984.0 kB 619.5 kB/s eta 0:00:02
   ------- ------------------------------ 204.8/984.0 kB 655.1 kB/s eta 0:00:02
   -------- ----------------------------- 215.0/984.0 kB 570.5 kB/s eta 0:00:02
   ---------- --------------------------- 266.2/984.0 kB 606.6 kB/s eta 0:00:02
   ----------- -------------------------- 286.7/984.0 kB 

In [45]:
import math
import numpy as np
import sys
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Iterable
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from orientation import Orientation
from complex import Complex

In [20]:
# Load the CSV file
file_path = 'data/imu_cal.csv'
data = pd.read_csv(file_path)

# Calculate the averages of the 6 IMU measurements
imu_columns = ['gyro_uncal_x', 'gyro_uncal_y', 'gyro_uncal_z', 'accel_uncal_x', 'accel_uncal_y', 'accel_uncal_z']
averages = data[imu_columns].mean()

# Print the average values
for column, avg in averages.items():
	print(f"Average {column}: {avg}")

Average gyro_uncal_x: 0.0009009989173953272
Average gyro_uncal_y: 0.0005190715405440818
Average gyro_uncal_z: 0.0014383745604558133
Average accel_uncal_x: -0.11726119600677827
Average accel_uncal_y: 0.06609904019018262
Average accel_uncal_z: 9.855918517874644


In [26]:
# Load the CSV file
file_path = 'data/imu_1.csv'
df = pd.read_csv(file_path)

df["gyro_x"] = (df["gyro_uncal_x"] - averages["gyro_uncal_x"])
df["gyro_y"] = (df["gyro_uncal_y"] - averages["gyro_uncal_y"])
df["gyro_z"] = (df["gyro_uncal_z"] - averages["gyro_uncal_z"])

In [27]:
def update(dt: float, gyro: np.ndarray) -> Orientation:
    """
    Update the orientation quaternion based on gyroscope data.
    """
    global orientation
    norm = np.linalg.norm(gyro)
    # avoid division by zero
    if norm > 0 and norm < 1e-9:
        norm = 1e-9
    elif norm < 0 and norm < -1e-9:
        norm = -1e-9
    orientation *= Orientation.from_axis_angle(
        gyro / norm,
        dt * norm
    )
    return orientation

# Ensure expected columns exist
expected = {"time_ms", "gyro_x", "gyro_y", "gyro_z"}
if not expected.issubset(df.columns):
    missing = expected - set(df.columns)
    raise ValueError(f"Missing required columns: {', '.join(missing)}")

# Compute dt (s) between rows, first row gets dt=0
df["dt"] = df["time_ms"].diff().fillna(0) * 1e-3

# Allocate columns for quaternion components
quat_cols: List[str] = ["w", "x", "y", "z"]
for c in quat_cols:
    df[c] = 0.0

global orientation
orientation = Orientation()

for idx, row in df.iterrows():
        dt  = float(row["dt"])
        gyro = np.array([float(row["gyro_x"]),
                        float(row["gyro_y"]),
                        float(row["gyro_z"])])
        q = update(dt, gyro)
        df.loc[idx, quat_cols] = q.quaternion

df_out = df[["time_ms", *quat_cols]]
df_out.to_csv("data/ori_1.csv", index=False)